In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-bright')

import pickle
from IPython.display import display

In [52]:
# %%memit pass
%%timeit 

df = pd.DataFrame({'k': [1,1,2,2,3,3,3], 'v': [10,20,40,30,10,11,12], 'type':[0,0,0,2,0,2,2]})
print df
df.index = df.k
df = df.drop('k',1)

def myf(df):
    temp_df = df.loc[df['type'] == 2]
    if temp_df.empty:
        return df.iloc[0] # [0:1]
    else:
        return temp_df.iloc[0]

df = df.groupby(df.index).apply(myf) 

# df

SyntaxError: invalid syntax (<ipython-input-52-f7beae2e1cd1>, line 2)

In [3]:
output = temp_df.iloc[-1]
output['v'] = (temp_df['type']*temp_df['v']).sum() 
output

NameError: name 'temp_df' is not defined

In [45]:
%%memit
# custom groupby
df = pd.DataFrame({'k': [1,1,2,2,3,3,3], 'v': [10,20,40,30,10,11,12], 'type':[0,0,0,2,0,2,2]})
df.index = df.k
df = df.drop('k',1)

def myf(df):
    temp_df = df.loc[df['type'] == 2]
    if temp_df.empty:
        return df.iloc[0] # [0:1]
    else:
        return temp_df.iloc[0]

df = df.groupby(df.index).apply(myf) 

print '1'

yay
1
yay
1
peak memory: 94.61 MiB, increment: 0.42 MiB


In [44]:
import memory_profiler
memory_profiler.__file__

'/Users/sidz/anaconda2/lib/python2.7/site-packages/memory_profiler.pyc'

In [ ]:
%%memit